In [1]:
import argparse

from tqdm import tqdm

In [2]:
%load_ext autoreload
%autoreload 2

from load_data import load_data
from model import Optimizer, nihgcn
from myutils import *
from sampler import NewSampler
from sklearn.model_selection import KFold

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7fbc04f4b490>>
Traceback (most recent call last):
  File "/scratch.global/inoue019/conda-envs/myenv/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 


In [3]:
class Args:
    def __init__(self):
        self.device = torch.device(
            "cuda" if torch.cuda.is_available() else "cpu"
        )  # cuda:number or cpu
        self.data = "nci"  # Dataset{gdsc or ccle}
        self.lr = 0.001  # the learning rate
        self.wd = 1e-5  # the weight decay for l2 normalizaton
        self.layer_size = [1024, 1024]  # Output sizes of every layer
        self.alpha = 0.25  # the scale for balance gcn and ni
        self.gamma = 8  # the scale for sigmod
        self.epochs = 1000  # the epochs for model


args = Args()

In [4]:
res, drug_finger, exprs, null_mask, pos_num = load_data(args)
cell_sum = np.sum(res, axis=1)
drug_sum = np.sum(res, axis=0)

target_dim = [
    # 0,  # Cell
    1  # Drug
]

load nci


In [5]:
def nihgcn_new(
    cell_exprs,
    drug_finger,
    res_mat,
    null_mask,
    target_dim,
    target_index,
    evaluate_fun,
    args,
    seed,
):

    sampler = NewSampler(res_mat, null_mask, target_dim, target_index, seed)
    model = nihgcn(
        sampler.train_data,
        cell_exprs=cell_exprs,
        drug_finger=drug_finger,
        layer_size=args.layer_size,
        alpha=args.alpha,
        gamma=args.gamma,
        device=args.device,
    )
    opt = Optimizer(
        model,
        sampler.train_data,
        sampler.test_data,
        sampler.test_mask,
        sampler.train_mask,
        evaluate_fun,
        lr=args.lr,
        wd=args.wd,
        epochs=args.epochs,
        device=args.device,
    )
    true_data, predict_data = opt()
    return true_data, predict_data

In [7]:
from joblib import Parallel, delayed
from tqdm import tqdm

n_kfold = 1
n_jobs = 50  # 並列数


def process_iteration(dim, target_index, seed, args):
    """各反復処理をカプセル化した関数"""
    if dim:
        if drug_sum[target_index] < 10:
            return None, None
    else:
        if cell_sum[target_index] < 10:
            return None, None

    fold_results = []
    for fold in range(n_kfold):
        true_data, predict_data = nihgcn_new(
            cell_exprs=exprs,
            drug_finger=drug_finger,
            res_mat=res,
            null_mask=null_mask,
            target_dim=dim,
            target_index=target_index,
            evaluate_fun=roc_auc,
            args=args,
            seed=seed,
        )
        fold_results.append((true_data, predict_data))

    return fold_results


# 並列処理の実行
true_data_s = pd.DataFrame()
predict_data_s = pd.DataFrame()

for dim in target_dim:
    # 全タスクを事前に生成
    tasks = [
        (dim, target_index, seed, args)
        for seed, target_index in enumerate(np.arange(res.shape[dim]))
    ]

    # 並列実行（プログレスバー付き）
    results = Parallel(n_jobs=n_jobs, verbose=0, prefer="threads")(
        delayed(process_iteration)(*task)
        for task in tqdm(tasks, desc=f"Processing dim {dim}")
    )

    # 結果の結合
    for fold_results in results:
        if fold_results is None:
            continue
        for true_data, predict_data in fold_results:
            true_data_s = pd.concat(
                [true_data_s, translate_result(true_data)],
                ignore_index=True,
                copy=False,  # メモリ節約のため
            )
            predict_data_s = pd.concat(
                [predict_data_s, translate_result(predict_data)],
                ignore_index=True,
                copy=False,
            )


Processing dim 1:   0%|          | 0/1005 [00:00<?, ?it/s]

epoch:   0 loss:0.702110 auc:0.4601
epoch:   0 loss:0.703279 auc:0.4856
epoch:   0 loss:0.699967 auc:0.4274
epoch:   0 loss:0.699514 auc:0.4905
epoch:   0 loss:0.701154 auc:0.5673
epoch:   0 loss:0.698611 auc:0.7056
epoch:  20 loss:0.492896 auc:0.8627
epoch:  20 loss:0.497765 auc:0.6914
epoch:  20 loss:0.497619 auc:0.6707
epoch:  20 loss:0.487341 auc:0.7394
epoch:  20 loss:0.490977 auc:0.6830
epoch:  40 loss:0.403531 auc:0.8089
epoch:  40 loss:0.406209 auc:0.7392
epoch:  40 loss:0.407665 auc:0.6274
epoch:   0 loss:0.701689 auc:0.3284
epoch:   0 loss:0.701844 auc:0.5672
epoch:   0 loss:0.701710 auc:0.5196
epoch:   0 loss:0.701078 auc:0.4932
epoch:   0 loss:0.700341 auc:0.6329
epoch:   0 loss:0.698686 auc:0.8053
epoch:   0 loss:0.701420 auc:0.6537
epoch:   0 loss:0.699955 auc:0.4313
epoch:   0 loss:0.702149 auc:0.7111
epoch:   0 loss:0.697911 auc:0.4912
epoch:   0 loss:0.696305 auc:0.6042
epoch:   0 loss:0.698461 auc:0.6750
epoch:  20 loss:0.504077 auc:0.6021
epoch:   0 loss:0.698429 auc

KeyboardInterrupt: 

epoch: 480 loss:0.256841 auc:0.5918
epoch: 400 loss:0.266207 auc:0.6194
epoch:  60 loss:0.358068 auc:0.8088
epoch: 600 loss:0.252975 auc:0.5770
epoch: 560 loss:0.254631 auc:0.6364
epoch: 500 loss:0.256121 auc:0.6073
epoch: 200 loss:0.286707 auc:0.6494
epoch: 620 loss:0.252370 auc:0.5807
epoch: 580 loss:0.252914 auc:0.6065
epoch: 420 loss:0.260928 auc:0.7383
epoch: 300 loss:0.269899 auc:0.2570
epoch: 520 loss:0.256354 auc:0.5784


In [ ]:
true_data_s.to_csv(f"new_drug_true_{args.data}.csv")
predict_data_s.to_csv(f"new_drug_pred_{args.data}.csv")